<a href="https://colab.research.google.com/github/Xi123580/Product-Information-Conversion/blob/main/Product_Data_Converter_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
# from time import perf_counter
from google.colab import files
import io
# import time
import datetime
import os


pd.set_option('max_columns', None)

# 1. read file
path = 'https://github.com/Xi123580/Product-Information-Conversion/blob/main/%E7%94%A2%E5%93%81%E8%B3%87%E8%A8%8A_%E5%8E%9F%E5%A7%8B%E8%B3%87%E6%96%99_%E6%AC%84%E4%BD%8D%E8%BD%89%E6%8F%9B%E8%A1%A8_%E8%B3%87%E6%96%99%E8%BD%89%E6%8F%9B%E8%A1%A8_20220421_1.xlsx?raw=true'
df_rawData = pd.read_excel(path, sheet_name = 0)
print(df_rawData.head().to_string())


             IPROD                                     IDESC                IMNOT1       IDRAW  SAPSBU  ILIST IMFRMC IFCI IMCMTH SAPNOR IPURC
0      618003.1566               Spectinomycin Hydrochloride             觀黴素鹽酸鹽標準品  22189-32-8   781.0  10600      .    A     M2      Y   M2D
1     1.00327.5000  HYDROCHLORIC ACID IN ETHANOL FOR MICROSC          鹽酸標準液配製在乙醇溶液         NaN   649.0   8600    IVD    S    NaN    NaN   M2B
2  EPV0045000.1566                  Vancomycin hydrochloride                鹽酸萬古黴素   1404-93-9   823.0   9800     FD    A    NaN    NaN   M2D
3    P3850000.1566  Pseudoephedrine hydrochloride *Precursor  鹽酸假麻黃素標準品,第四級管制先驅原料藥         NaN   823.0  12400    DEA    A    NaN    NaN   M2D
4    Y0000218.1566       Ondansetron hydrochloride dihydrate                鹽酸昂丹司瓊         NaN   823.0   9600     NC    A    NaN    NaN   M2D


In [2]:
# Page: 3~8
def initial_sheet(Page:int):
  df = pd.read_excel(path, sheet_name = Page, usecols = "A, B")
  df.rename(columns={'原始資訊': 'Keys', '網頁呈現字眼': 'Values'}, inplace=True)
  df = df.drop(0)
  df = df.reset_index()
  df = df.drop(columns="index")

  column_SAPSBU = []
  column_Product = []
  ConvertDict = {}
  for i in range(len(df.values[:])):
    column_SAPSBU.append(df.values[i][0])
    column_Product.append(df.values[i][1])
    ConvertDict[column_SAPSBU[i]] = column_Product[i]
  return(ConvertDict)

In [3]:
Rule_SAPSBU = initial_sheet(2)
Rule_IMFRMC = initial_sheet(3)
Rule_IMCMTH = initial_sheet(4)
Rule_IFCI = initial_sheet(5)
Rule_SAPNOR = initial_sheet(6)
Rule_IPURC = initial_sheet(7)
print(Rule_IFCI)
print(Rule_IMCMTH)

{'A': '空運', 'AIR': '空運', 'C': '空運', 'L': '請洽默克窗口', 'N': '請洽默克窗口', nan: '請洽默克窗口', 'P': '空運', 'PAX': '空運', 'S': '海運', 'SEA': '海運', 'Y': '請洽默克窗口'}
{7: '已停產', 99: '需法規審核', 'M1': '已停產', 'M2': '已停產', 'YY': '已停產'}


In [4]:
# upload file ( .xlsx )
print("請上傳Excel檔 (副檔名須為.xlsx): ")
uploaded = files.upload()
# Start_1 = perf_counter()
filename = next(iter(uploaded))
uploaded = pd.read_excel(io.BytesIO(uploaded.get(f'{filename}')), dtype=str)
print(uploaded.head(3).to_string())

請上傳Excel檔 (副檔名須為.xlsx): 


Saving IIM_Product_List (1).xlsx to IIM_Product_List (1).xlsx
         IPROD                                     IDESC                                    IMNOT1      IDRAW SAPSBU ILIST IMFRMC IFCI IMCMTH SAPNOR IPURC
0  185450-100G  Phenol, unstabilized, ReagentPlus, >=99%  GUS: PHENOL, UNSTABILIZED, REAGENTPLUS(R   108-95-2    774  2900    NaN    A    NaN    NaN   M2A
1    I2215-25G                            5-INDANOL, 99%                       GUS: 5-INDANOL, 99%  1470-94-6    774  3200      .    A    NaN    NaN   M2A
2  110388-25ML                    BUTYLCYCLOHEXANE, 99+%                    BUTYLCYCLOHEXANE, 99+%  1678-93-9    774  2600      .    A    NaN    NaN   M2A


In [5]:
# 3. build table
# Start_2 = perf_counter()
New_df = uploaded.copy()

print(New_df.keys())
print(New_df.shape)
Number_columns = New_df.shape[1]
Number_row = New_df.shape[0]
New_df = New_df.fillna(0)

# figure out table
for col in range(Number_columns):
  Col_Name = df_rawData.keys()[col]
  if Col_Name == "SAPSBU":
    # New_df[Col_Name] = New_df[Col_Name].apply(int)
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_SAPSBU:
        x = Rule_SAPSBU[data]
        New_df.at[row, Col_Name] = x
      else:
        New_df.at[row, Col_Name] = "-"
  elif Col_Name == "IMFRMC":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IMFRMC:
        x = Rule_IMFRMC[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "無管制_但請再確認注意事項"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "IFCI":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IFCI:
        x = Rule_IFCI[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "-"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "IMCMTH":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IMCMTH:
        x = Rule_IMCMTH[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "-"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "SAPNOR":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_SAPNOR:
        x = Rule_SAPNOR[data]
        New_df.at[row, Col_Name] = x
      else:
        New_df.at[row, Col_Name] = "-"
  elif Col_Name == "IPURC":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IPURC:
        x = Rule_IPURC[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "Blue"
      else:
        New_df.at[row, Col_Name] = "Blue"    
  else :
    pass

# Change Title
# New_df = New_df.rename(Translation, axis=1)

print(New_df.head().to_string())

Index(['IPROD', 'IDESC', 'IMNOT1', 'IDRAW', 'SAPSBU', 'ILIST', 'IMFRMC',
       'IFCI', 'IMCMTH', 'SAPNOR', 'IPURC'],
      dtype='object')
(277900, 11)
         IPROD                                     IDESC                                    IMNOT1       IDRAW                           SAPSBU ILIST         IMFRMC IFCI IMCMTH SAPNOR IPURC
0  185450-100G  Phenol, unstabilized, ReagentPlus, >=99%  GUS: PHENOL, UNSTABILIZED, REAGENTPLUS(R    108-95-2  LSC_774 Chemical Synthesis - RC  2900  無管制_但請再確認注意事項   空運      -      -   Red
1    I2215-25G                            5-INDANOL, 99%                       GUS: 5-INDANOL, 99%   1470-94-6  LSC_774 Chemical Synthesis - RC  3200  無管制_但請再確認注意事項   空運      -      -   Red
2  110388-25ML                    BUTYLCYCLOHEXANE, 99+%                    BUTYLCYCLOHEXANE, 99+%   1678-93-9  LSC_774 Chemical Synthesis - RC  2600  無管制_但請再確認注意事項   空運      -      -   Red
3    661384-1G  2-Iodoxybenzoic acid, stabilized (45 wt.  2-IODOXYBENZOIC ACID, STABILI

In [11]:
# 4. save file as .csv and output.

New_df.to_csv(path_or_buf = "ProductData.csv", sep = ",", na_rep = "",
      index = False, encoding = "utf-8-sig")

Today = datetime.date.today()
os.rename("/content/ProductData.csv", f"ProductData_{Today}.csv")  
files.download(f"ProductData_{Today}.csv")

print("Finish")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Finish
